# Definición de Librería

In [ ]:
import tweepy
import json
import pandas as pd
import time
import GetOldTweets3 as got


In [ ]:
%run Metodos_Propios.ipynb
%run Gestion_Ficheros.ipynb


In [ ]:
pd.set_option('display.max_rows', 1000)

In [ ]:
#Definción de variables
df_followers_id = pd.DataFrame()
df_followers_info = pd.DataFrame()


# Definición de las claves para acceder a Twitter

In [ ]:
#Variable de autentificación en Twitter
OAUTH_TOKEN='1204477616398503936-7Z5WSIa0490S7HbIoPrDzosdA8aJKb'
OAUTH_SECRET='OCHWxZGQlrnaoNx4corTWHNZPrtGtItteYnow7nMDhtON'
CONSUMER_KEY='IRuah0diiQDx7ayuHaFmCOvTq'
CONSUMER_SECRET='FkYZE9vZXWMgWvovRjcqQaixPleh6ADXdD3XU05ZsIkscSFzXd'

In [ ]:
auth = tweepy.OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
auth.set_access_token(OAUTH_TOKEN,OAUTH_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, timeout=3600)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns',None)
pd.set_option('display.width',None)
pd.set_option('max_colwidth',-1)

# Extracción de los datos de la cuenta

In [ ]:
def extract_info_account():
    item = api.get_user('@elcorteingles')
    data_account=[]
    account={}
    account["name"]=item.name
    account["id"]=item.id
    account["screen name"]=item.screen_name
    account["friends_count"]=item.friends_count
    account["followers_count"]=item.followers_count
    data_account.append(account)
    df_account_info = pd.DataFrame(data_account)
    return df_account_info


# Followings

# Extracción de las cuentas Followings

In [ ]:
# INFORMACION DE LIMITACION: https://developer.twitter.com/en/docs/basics/rate-limits
def get_following_ids(api, twitter_account='@elcorteingles'):
    while True:
        try:
            for page in tweepy.Cursor(api.friends_ids, id=twitter_account, count=5000).pages():
                following_ids.extend(page)
                #print(len(following_ids))
            break
        except tweepy.TweepError as e:
            time.sleep(60*15)
            continue
    return following_ids


# Extracción información de las cuentas Followings

In [ ]:
def get_info_following(api,following_limit):
    data = []
    for followings in following_limit:
        following_obj=api.lookup_users(user_ids=followings) 
        for user in following_obj:
            following={}
            following["id"]=user.id
            following["name"]=user.name
            following["screen_name"]=user.screen_name
            following["created_at"]=user.created_at
            following["location"]=user.location
            following["followers_count"]=user.followers_count
            following["friends_count"]=user.friends_count
            following["description"]=user.description
            data.append(following)
    return data


In [ ]:
def saveFile():
    file= File_Manage()
    fl_account_info = 'CSVs/info_el_corte_ingles.csv'
    file.setFile(fl_account_info)
    file.writeCSV(df_account_info,file.getFile())
    fl_following_id = 'CSVs/following_id_el_corte_ingles.csv'
    file.setFile(fl_following_id)
    file.writeCSV(df_following_id,file.getFile())
    fl_following_info = 'CSVs/following_info_el_corte_ingles.csv'
    file.setFile(fl_following_info)
    file.writeCSV(df_following_info,file.getFile())
